<a href="https://colab.research.google.com/github/RunzeCheng/SemAIGCtransceiver/blob/main/semaigc_code_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project-SemAIGC
# Author: Runze Cheng
# University of Glasgow


In [ ]:
!pip install --quiet --upgrade diffusers transformers scipy mediapy accelerate

## Import models and packages

In [ ]:


import os
import cv2
import torch
import numpy as np

from torch import autocast
from tqdm.auto import tqdm
from pathlib import Path
from huggingface_hub import HfApi
from PIL import Image, ImageDraw
from diffusers.utils import load_image
from google.colab import output
from huggingface_hub import notebook_login
import torch.nn.functional as F

from diffusers import StableDiffusionPipeline, AutoencoderKL, UNet2DConditionModel, PNDMScheduler, LMSDiscreteScheduler, DDIMScheduler, DDPMScheduler
from transformers import CLIPTextModel, CLIPTokenizer, AutoImageProcessor, UperNetForSemanticSegmentation

# from base64 import b64encode #image to video and save
# from Ipython.display import HTML
"""
stablediffusionPipline: uses the PNDMScheduler by default
AutoencoderKL: autoencoder model that can decode the latents into image
UNet2DConditionModel: UNet model for denoising perocess
PNDMScheduler, LMSDiscreteScheduler, DDIMScheduler, DDPMScheduler: different scheduler for interference  #The used scheduler in paper is DDPM, however, according to our test, DDIM performance beter.
CLIPTextModel,CLIPTokenizer: Extract text as token
AutoImageProcessor, UperNetForSemanticSegmentation: Extract image segmentation
"""

In [ ]:
# """Geonmo/laion-aesthetic-predictor"""
# import pytorch_lightning as pl
# import torch.nn as nn
# import clip
# from PIL import Image, ImageFile
# import gradio as gr
from google.colab import drive
drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive/Dataset_Sem")
filepath="/content/drive/MyDrive/Dataset_Sem/"

Set device and huggingface account


In [ ]:
device = 'cuda'

output.enable_custom_widget_manager()
notebook_login()

# Load Semantic Processing Models for Modules in SemAIGC

Channel-Aware Semantic noise schedular

In [ ]:
import torch
import numpy as np
import math
import sympy as sy
from scipy.optimize import bisect



In [ ]:
#This is the simplified version of schedular for fine-tuning module, just for testing.
# The channel-noise adjustable scheduler (version 2.0) can be found in the file AIGCscheduler.py
# Since we didn't specifically refine this, the image could come out green.
#It is recommand to use original DDIM schedular, and consider adjusting the sigma value of channel caused channel noise, still works.

class FFschedular:
  '''Schedular for the fine-tuning module'''

  def __init__(self,timesteps, R_step, start=0.0001, end=0.02, init_gamma=0.005, n_sigma):
    self.timesteps = timesteps
    self.R_step = R_step
    self.start = start
    self.end = end
    self.alpha_sch = []
    self.init_gamma = init_gamma
    self.n_sigma = n_sigma
  def linear_beta_schedule(self):
    """'test with defualt schadular'"""
    self.beta_sch = torch.linspace(self.start, self.end, self.timesteps)
    self.alpha_sch = 1 - self.beta_sch
    return self.alpha_sch

  def gamma_schedule(self):
    prod_alpha=[]
    alpha_table = self.linear_beta_schedule()
    print(alpha_table)
    alpha_table=alpha_table.tolist()
    print(alpha_table[0])
    for t in range(self.R_step):
      if t == 0:
        alpha_sqrt = 1
      else:
        alpha_sqrt = math.sqrt(alpha_table[t])
      prod_alpha.append(1)
      prod_alpha = [x * alpha_table[t] for x in prod_alpha]
      # prod_alpha = prod_alpha*alpha_table[t]
    # gamma_anumber = list(range(self.R_step))

    # multi_a_r = [x * math.sqrt(y) for x, y in zip(prod_alpha, gamma_anumber)]
    # gamma_step = (1-(sum(prod_alpha)*math.sqrt(self.init_gamma)))/sum(multi_a_r)


    self.gamma_sch = torch.linspace(self.init_gamma, self.init_gamma+self.R_step*gamma_step, self.R_step)*self.n_sigma
    return self.gamma_sch
schedular_r= FFschedular(20,20)
a= schedular_r.gamma_schedule()

print(a)

  # def get_index_from_list(self,vals, t, x_shape):
  #   """
  #   Returns a specific index t of a passed list of values vals
  #   while considering the batch dimension.
  #   """
  #   batch_size = t.shape[0]
  #   out = vals.gather(-1, t.cpu())
  #   return out.reshape(batch_size, *((1,) * (len(x_shape) - 1))).to(t.device)

#   def forward_diffusion_sample(x_0, t, device="cpu"):
#     """
#     Takes an image and a timestep as input and
#     returns the noisy version of it
#     """
#     noise = torch.randn_like(x_0)
#     sqrt_alphas_cumprod_t = get_index_from_list(sqrt_alphas_cumprod, t, x_0.shape)
#     sqrt_one_minus_alphas_cumprod_t = get_index_from_list(
#         sqrt_one_minus_alphas_cumprod, t, x_0.shape
#     )
#     # mean + variance
#     return sqrt_alphas_cumprod_t.to(device) * x_0.to(device) \
#     + sqrt_one_minus_alphas_cumprod_t.to(device) * noise.to(device), noise.to(device)


#   # Define beta schedule
#   T = 300
#   betas = linear_beta_schedule(timesteps=T)

#   # Pre-calculate different terms for closed form
#   alphas = 1. - betas
#   alphas_cumprod = torch.cumprod(alphas, axis=0)
#   alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)
#   sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
#   sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
#   sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod)
#   posterior_variance = betas * (1. - alphas_cumprod_prev) / (1. - alphas_cumprod)

In [ ]:
# # from sympy import symbols, sqrt, solve

# # Define symbols
# a, b, x = sy.symbols('a b x')
# a_list=list(range(10))
# b_list=list(range(10))
# # Define the equation
# equation = sy.Sum(a * sy.sqrt(0.01 + b) - 1, )

# # Solve the equation for x
# solutions = sy.solve(equation.subs(a, a_list[1]).subs(b, b_list[1]), x)

# # Display the solutions
# print("Solutions for x:",solutions)
# for solution in solutions:
#     print(solution.evalf())

In [ ]:
# import numpy as np
# from scipy.optimize import bisect

# def equation(x, a, b):
#     total = 0
#     for i in range(1, 9):
#         arg_sqrt = b + i * x
#         if arg_sqrt < 0:
#             return np.inf  # Return infinity if the argument of the square root is negative
#         total += a[i-1] * np.sqrt(arg_sqrt)
#     return total - 1

# a = [1, 2, 3, 4, 5, 6, 7, 8]
# b = 0.001

# # Define the bounds for the bisection method
# lower_bound = -1e6
# upper_bound = 1e6

# # Use the bisection method to find the root of the equation
# solution = bisect(equation, lower_bound, upper_bound, args=(a, b))

# print("Solution for x:", solution)

Semantic extraction module:

In [ ]:
# Load the tokenizer/text encoder model for semantic extraction module.

tokenizer = CLIPTokenizer.from_pretrained('openai/clip-vit-large-patch14')
text_encoder = CLIPTextModel.from_pretrained('openai/clip-vit-large-patch14')
text_encoder = text_encoder.to(device)

# Load the image Autoencoder for semantic extraction module.

vae_E = AutoencoderKL.from_pretrained('CompVis/stable-diffusion-v1-4', subfolder='vae', use_auth_token=True)
vae_E = vae_E.to(device)


Semantic-generating module:

In [ ]:
# Load the Pre-trained UNet and it corresponding scheduler for semantic-generating module.

unet_E = UNet2DConditionModel.from_pretrained(
    'CompVis/stable-diffusion-v1-4', subfolder='unet', use_auth_token=True)
unet_E = unet_E.to(device)

scheduler_E = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule='scaled_linear', num_train_timesteps=1000)
# scheduler_E = DDPMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule='scaled_linear', num_train_timesteps=1000)

Fine-tuning module:

In [ ]:
# Load the Pre-trained UNet for fine-tuning module.

unet_D = UNet2DConditionModel.from_pretrained(
    'CompVis/stable-diffusion-v1-4', subfolder='unet', use_auth_token=True)
unet_D = unet_D.to(device)



Autoencoder decoder module

In [ ]:
# Load the image decoder for Autoencoder decoder module.

vae_D = AutoencoderKL.from_pretrained(
    'CompVis/stable-diffusion-v1-4', subfolder='vae', use_auth_token=True)
vae_D = vae_D.to(device)


# Semantic process functions

In [ ]:
class SemanticProcessing:
  def __init__(self,promot,snr = 0, seedsize=32, height = 512, width = 512, guidence_scale = 7.5, Encoder_steps = 15, Decoder_steps = 5, ideal_channel=False):

    self.promot = promot
    self.snr = snr
    self.seedsize = seedsize
    self.height = height
    self.width = width
    self.guidence_scale = guidence_scale
    self.num_inference_steps = Encoder_steps + Decoder_steps
    self.start_step = Encoder_steps
    self.ideal_channel = ideal_channel
    self.Pn_2 = []
    # self.text_semantics = []
    # self.latent_1 = [] # Guassian noise
    # self.latents_2 = [] # Transmitter generated semantics
    # # self.all_latents_2 = []
    # self.latents_3 = [] # Semantics with channel noise-caused semantic noise
    # self.latents_4 = [] # Decoder fine-turned semantic
  # def ExtractingImageSemantic():

  def GenerateInitGnoise(self,batch_size=1):
    generator = torch.manual_seed(self.seedsize)
    # self.latents_1 = torch.randn(batch_size,seedsize=32)
    semantics = torch.randn((batch_size,unet_D.in_channels,self.height//8,self.width//8),generator=generator)
    semantics=semantics.to(device)
    return semantics

  def ExtractingTextSemantic(self,batch_size=1):
    # convert text to text token with condition
    text_token = tokenizer(self.promot, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")

    with torch.no_grad():
      text_sem = text_encoder(text_token.input_ids.to(device))[0]

    # generate unconditional token
    max_length = text_token.input_ids.shape[-1]
    uncon_token = tokenizer([""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt")

    with torch.no_grad():
      uncon_sem = text_encoder(uncon_token.input_ids.to(device))[0]

    # concatenates the given token sequence along a new dimension
    text_semantics = torch.cat([uncon_sem, text_sem])

    return text_semantics


  def DiffusionGenerateSemantic(self,semantics,text_semantics,encoder_denoise_step=5,return_encoder_latents=False):
    semantics = semantics.to(device)
    scheduler_E.set_timesteps(self.num_inference_steps)
    semantics = semantics * scheduler_E.init_noise_sigma
    # latent_hist=[]
    # for t in tqdm(scheduler_E.timesteps):
    for t in scheduler_E.timesteps:
      latent_model_input = torch.cat([semantics] * 2)
      latent_model_input = scheduler_E.scale_model_input(latent_model_input, t)
    # predict noise distribution
      with torch.no_grad():
        noise_pred = unet_E(latent_model_input,t,encoder_hidden_states = text_semantics).sample

      noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
      noise_pred = noise_pred_uncond + self.guidence_scale * (noise_pred_text - noise_pred_uncond)
      semantics = scheduler_E.step(noise_pred, t, semantics).prev_sample
      if t == scheduler_E.timesteps[-encoder_denoise_step]:
        encoder_semantics = semantics
      # latent_hist_2.append(self.latents_2)
    if not return_encoder_latents:
      return semantics
    else:
      return encoder_semantics
    # self.all_latents_2 = torch.cat(latent_hist_2, dim=0)
    # return self.all_latents_2


  def ChannelNoise(self,semantics,gain=1):
    #AWGN Channel
    h_latents = semantics.shape[2]
    w_latents = semantics.shape[3]
    Ps = torch.sum(torch.pow(semantics,2))/h_latents/w_latents
    Pn = Ps/(np.power(10,self.snr/10))
    self.Pn_2 = torch.sqrt(Pn).to("cpu")
    noise_channel=torch.randn_like(semantics).uniform_(0,1)*self.Pn_2

    semantics=semantics.to(device)
    scheduler_E.set_timesteps(self.num_inference_steps)
    if self.start_step > 0:
      start_timestep = scheduler_E.timesteps[self.start_step]
      start_timesteps = start_timestep.repeat(semantics.shape[0]).long()

      noise = torch.randn_like(semantics)
      semantics = scheduler_E.add_noise(semantics, noise, start_timesteps)
      if not self.ideal_channel:
        semantics = semantics + noise_channel/2
        # print("<<<<<<<<",self.snr,">>>>>>>>")
      else:
        print("<<<<<<<<Ideal Channel>>>>>>>>")
    return semantics

  def FineTuning(self, semantics, text_semantics, guidance_scale=7.5, return_all_latents=False):
    semantics = semantics.to(device)
    with autocast('cuda'):
      # for i, t in tqdm(enumerate(scheduler_E.timesteps[self.start_step:])):
      for i, t in enumerate(scheduler_E.timesteps[self.start_step:]):
        # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
        latent_model_input = torch.cat([semantics] * 2)

        # predict the noise residual
        with torch.no_grad():
          noise_pred = unet_D(latent_model_input, t, encoder_hidden_states=text_semantics).sample

        # perform guidance
        noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
        noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)*self.Pn_2

        # compute the previous noisy sample x_t -> x_t-1
        semantics = scheduler_E.step(noise_pred, t, semantics).prev_sample
    if not return_all_latents:
      return semantics



  def DecodeImage(self,semantics,I_names):

    semantics = 1 / 0.18215 * semantics
    names=locals()
    with torch.no_grad():
      image = vae_D.decode(semantics).sample
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (image * 255).round().astype("uint8")
    Out_images = [Image.fromarray(image) for image in images]
    Out_images[0].save(filepath+I_names)#'image_out.png'
    Out_images[0]

  def OutputImage(self,I_names,UEoutput=False):
    image_semantic_E = self.GenerateInitGnoise()
    text_semantic_E = self.ExtractingTextSemantic()
    image_semantic_E = self.DiffusionGenerateSemantic(image_semantic_E,text_semantic_E)
    if not UEoutput:
      self.DecodeImage(image_semantic_E,I_names)
      #output image in transmitter
    # else:
    #   image_semantic_D = self.ChannelNoise(image_semantic_E)+image_semantic_E
    #   self.all_latents_2 = image_semantic_D+noise
    #   image_semantic_D = self.DiffusionGenerateSemantic(unet=unet_E)
    #   self.DecodeImage(image_semantic_D,I_names)


### Fine-Tune Diffusion UNet ** (Optional)

In [ ]:
'''Details see
#https://github.com/cloneofsimo/lora
#https://github.com/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb
#https://github.com/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-finetuner.ipynb
'''


# Semantic-empowered AIGC transmission

In [ ]:
SemAIGC = SemanticProcessing("A cute furry cat",6, seedsize=30, Encoder_steps = 5, Decoder_steps = 15,) #28 #30

Image_semantics_0 = SemAIGC.GenerateInitGnoise() #Generated initial Guassian noise
# print(latents_0)
text_semantics = SemAIGC.ExtractingTextSemantic() #Text semantic information


Image_semantic_1 = SemAIGC.DiffusionGenerateSemantic(Image_semantics_0,text_semantics) #Image semantic information




In [ ]:
# def wgn(latents,h,w,snr):

#     Ps=np.sum(np.power(latents,2))/h/w
#     Pn=Ps/(np.power(10,snr/10))
#     print("Pn",Pn)
#     noise=np.random.randn(latents.shape).uniform_(0,1)*np.sqrt(Pn)
#     return noise
# def wgn_2(semantics,snr):
#     h_latents = semantics.shape[2]
#     w_latents = semantics.shape[3]
#     Ps = torch.sum(torch.pow(semantics,2))/h_latents/w_latents
#     Pn = Ps/(np.power(10,snr/10))
#     Pn_2 = torch.sqrt(Pn).to("cpu")
#     noise=torch.randn_like(semantics).uniform_(0,1)*Pn_2
#     return noise

In [ ]:
# Image_NON_SEM=Image_semantic_1+wgn_2(Image_semantic_1,-3)
# Image_NON = SemAIGC.DecodeImage(Image_NON_SEM,"Image_NON_SEM.png")

In [ ]:

Image_E = SemAIGC.DecodeImage(Image_semantic_1,"out_1.png")
print(Image_E)
image_c = cv2.imread("out_1.png")
height=512
width=512
channels=3
# Gaussian mean and variance
# snr = 6
# bandwidth = 0.0005 * 10**6  # 5 MHz
snr=-3
# # calculate
# noise_power = 10**(-snr / 10) * bandwidth
Ps=np.sum(np.power(image_c,2))/height/width
noise_power=Ps/(np.power(10,snr/10))
# generate Gaussian Noise matrixes/tensors

noise = np.random.normal(0, np.sqrt(noise_power), (height, width, channels))

print(noise)
# add noise to image
noisy_image = image_c + noise

print(noisy_image)
# avoid pixels of noisy images exceed 255
noisy_image = np.clip(noisy_image, 0, 255)
# save images
cv2.imwrite("noisy_img.png",noisy_image)


In [ ]:
Image_semantic_1_en = SemAIGC.DiffusionGenerateSemantic(Image_semantics_0,text_semantics,return_encoder_latents=True )

In [ ]:
Image_i = SemAIGC.DecodeImage(Image_semantics_0,"out_0.png")

Image_E = SemAIGC.DecodeImage(Image_semantic_1,"out_1.png")

Image_E_en = SemAIGC.DecodeImage(Image_semantic_1_en,"out_1_en.png")


In [ ]:
Image_semantic_noise = SemAIGC.ChannelNoise(Image_semantic_1,gain=10)
Image_n = SemAIGC.DecodeImage(Image_semantic_noise,"out_2.png")

Image_semantic_D = SemAIGC.FineTuning(Image_semantic_noise,text_semantics)
Image_D = SemAIGC.DecodeImage(Image_semantic_D,"out_3.png")

In [ ]:
for i in range(4):
  for SNR in range(-3,13,3):
      for a in range(1,19):
        b = 20-a

        SemAIGC = SemanticProcessing("A cute furry cat",SNR, seedsize=30, Encoder_steps = a, Decoder_steps = b,) #28 #30

        Image_semantics_0 = SemAIGC.GenerateInitGnoise() #Generated initial Guassian noise

        text_semantics = SemAIGC.ExtractingTextSemantic() #Text semantic information

        Image_semantic_1 = SemAIGC.DiffusionGenerateSemantic(Image_semantics_0,text_semantics) #Image semantic information

        Image_semantic_noise = SemAIGC.ChannelNoise(Image_semantic_1,gain=10)
        # Image_n = SemAIGC.DecodeImage(Image_semantic_noise,"out_2.png")

        Image_semantic_D = SemAIGC.FineTuning(Image_semantic_noise,text_semantics)

        Image_D = SemAIGC.DecodeImage(Image_semantic_D,"out_SNR"+str(SNR)+"_Encode"+str(a)+"_Decode"+str(b)+"("+str(i)+")"+".png")


In [ ]:
Ps_0 = torch.sum(torch.pow(Image_semantics_0,2))/64/64

Ps_1 = torch.sum(torch.pow(Image_semantic_1,2))/64/64

Ps_2 = torch.sum(torch.pow(Image_semantic_1_en,2))/64/64

Ps_3 = torch.sum(torch.pow(Image_semantic_noise,2))/64/64

Ps_4 = torch.sum(torch.pow(Image_semantic_D,2))/64/64

print(Ps_0,Ps_1,Ps_2,Ps_3,Ps_4)

In [ ]:
a=scheduler_E.timesteps[-1]
b=scheduler_E.timesteps[-2]
print(scheduler_E._get_variance(a,b))


In [ ]:
h_latents=Image_semantic_1.shape[2]
w_latents=Image_semantic_1.shape[3]
Ps=torch.sum(torch.pow(Image_semantic_1,2))/h_latents/w_latents
Pn=Ps/(np.power(10,6/10))
Pn=torch.sqrt(Pn).to("cpu")
print("Pn",Pn.device)
print("Image",Image_semantic_1.device)
aa=torch.randn(Image_semantic_1.shape).uniform_(0,1)
print("aa",aa.device)

noise=torch.randn(Image_semantic_1.shape).uniform_(0,1)*torch.sqrt(Pn)
noise=torch.randn(Image_semantic_1.shape).uniform_(0,1)*torch.sqrt(Pn)


In [ ]:

# if you changed the MLP architecture during training, change it also here:
class MLP(pl.LightningModule):
    def __init__(self, input_size, xcol='emb', ycol='avg_rating'):
        super().__init__()
        self.input_size = input_size
        self.xcol = xcol
        self.ycol = ycol
        self.layers = nn.Sequential(
            nn.Linear(self.input_size, 1024),
            #nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 128),
            #nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            #nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(64, 16),
            #nn.ReLU(),

            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.layers(x)

    def training_step(self, batch, batch_idx):
        x = batch[self.xcol]
        y = batch[self.ycol].reshape(-1, 1)
        x_hat = self.layers(x)
        loss = F.mse_loss(x_hat, y)
        return loss

    def validation_step(self, batch, batch_idx):
        x = batch[self.xcol]
        y = batch[self.ycol].reshape(-1, 1)
        x_hat = self.layers(x)
        loss = F.mse_loss(x_hat, y)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

def normalized(a, axis=-1, order=2):
    import numpy as np  # pylint: disable=import-outside-toplevel

    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2 == 0] = 1
    return a / np.expand_dims(l2, axis)

def load_models():
    model = MLP(768)

    device = "cuda" if torch.cuda.is_available() else "cpu"

    s = torch.load("sac+logos+ava1-l14-linearMSE.pth", map_location=device)

    model.load_state_dict(s)
    model.to(device)
    model.eval()

    model2, preprocess = clip.load("ViT-L/14", device=device)

    model_dict = {}
    model_dict['classifier'] = model
    model_dict['clip_model'] = model2
    model_dict['clip_preprocess'] = preprocess
    model_dict['device'] = device

    return model_dict

def predict(image):
    image_input = model_dict['clip_preprocess'](image).unsqueeze(0).to(model_dict['device'])
    with torch.no_grad():
        image_features = model_dict['clip_model'].encode_image(image_input)
        if model_dict['device'] == 'cuda':
            im_emb_arr = normalized(image_features.detach().cpu().numpy())
            im_emb = torch.from_numpy(im_emb_arr).to(model_dict['device']).type(torch.cuda.FloatTensor)
        else:
            im_emb_arr = normalized(image_features.detach().numpy())
            im_emb = torch.from_numpy(im_emb_arr).to(model_dict['device']).type(torch.FloatTensor)

        prediction = model_dict['classifier'](im_emb)
    score = prediction.item()

    return {'aesthetic score': score}

if __name__ == '__main__':
    print('\tinit models')

    global model_dict

    model_dict = load_models()

    inputs = [gr.inputs.Image(type='pil', label='Image')]

    outputs = gr.outputs.JSON()

    title = 'image aesthetic predictor'

    examples = ['example1.jpg', 'example2.jpg', 'example3.jpg']

    description = """
    # Image Aesthetic Predictor Demo
    This model (Image Aesthetic Predictor) is trained by LAION Team. See [https://github.com/christophschuhmann/improved-aesthetic-predictor](https://github.com/christophschuhmann/improved-aesthetic-predictor)
    1. This model is desgined by adding five MLP layers on top of (frozen) CLIP ViT-L/14 and only the MLP layers are fine-tuned with a lot of images by a regression loss term such as MSE and MAE.
    2. Output is bounded from 0 to 10. The higher the better.
    """

    article = "<p style='text-align: center'><a href='https://laion.ai/blog/laion-aesthetics/'>LAION aesthetics blog post</a></p>"

    with gr.Blocks() as demo:
        gr.Markdown(description)
        with gr.Row():
            with gr.Column():
                image_input = gr.Image(type='pil', label='Input image')
                submit_button = gr.Button('Submit')
            json_output = gr.JSON(label='Output')
        submit_button.click(predict, inputs=image_input, outputs=json_output)
        gr.Examples(examples=examples, inputs=image_input)
        gr.HTML(article)
    demo.launch()